In [2]:
import pandas as pd

In [3]:
info = pd.read_csv('SPL/Library_Collection_Inventory.csv') #, index_col=0)
info = info[['BibNum', 'ISBN', 'Title', 'Author', 'Publisher', 'PublicationYear', 'Subjects', 'ItemType']]
info = info[info['ItemType'].str.contains('bk')]
info['ISBN'] = info['ISBN'].str.split(',', expand=False)
# expand the ISBN column into multiple rows
info = info.explode('ISBN').reset_index(drop=True)
info.dropna(subset=['ISBN'], inplace=True)
info.head(3)

,BibNum,ISBN,Title,Author,Publisher,PublicationYear,Subjects,ItemType
0,3011076,1481425730,A tale of two friends / adapted by Ellie O'Rya...,"O'Ryan, Ellie","Simon Spotlight,",2014.,"Musicians Fiction, Bullfighters Fiction, Best ...",jcbk
1,3011076,1481425749,A tale of two friends / adapted by Ellie O'Rya...,"O'Ryan, Ellie","Simon Spotlight,",2014.,"Musicians Fiction, Bullfighters Fiction, Best ...",jcbk
2,3011076,9781481425735,A tale of two friends / adapted by Ellie O'Rya...,"O'Ryan, Ellie","Simon Spotlight,",2014.,"Musicians Fiction, Bullfighters Fiction, Best ...",jcbk


In [4]:
#book_set = pd.merge(book_set, info, left_on='isbn10', right_on='ISBN', how='left')
book_set = info.copy()
book_set.dropna(subset=['BibNum'], inplace=True)
book_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4176490 entries, 0 to 4451680
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   BibNum           int64 
 1   ISBN             object
 2   Title            object
 3   Author           object
 4   Publisher        object
 5   PublicationYear  object
 6   Subjects         object
 7   ItemType         object
dtypes: int64(1), object(7)
memory usage: 286.8+ MB


In [5]:
prefix = 'Checkouts_By_Title_Data_Lens_' # prefix for file names
suffix = '.csv' # suffix for file names
years = [2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
for year in years:
    df = pd.read_csv('SPL/' + prefix + str(year) + suffix)
    df = df[['BibNumber']]
    df['Checkouts'] = 1
    df = df.groupby('BibNumber').sum().reset_index()
    df.columns = ['BibNum', 'Checkouts_' + str(year)]
    book_set = pd.merge(book_set, df, on='BibNum', how='left')

C:\Users\sethj\AppData\Local\Temp\ipykernel_16500\3039891236.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('SPL/' + prefix + str(year) + suffix)


In [6]:
book_set.fillna(0, inplace=True)
book_set['first_pub'] = book_set['PublicationYear'].str.split(',').str[0].str.split('.').str[0].str.split('-').str[0].str.split(' ').str[0]

In [7]:
book_set['first_pub'] = pd.to_numeric(book_set['first_pub'], errors='coerce')

In [8]:
book_set['years since pub'] = 2018 - book_set['first_pub']

In [9]:
book_set['checkouts since pub'] = book_set['Checkouts_2005'] + book_set['Checkouts_2006'] + book_set['Checkouts_2007'] + book_set['Checkouts_2008'] + book_set['Checkouts_2009'] + book_set['Checkouts_2010'] + book_set['Checkouts_2011'] + book_set['Checkouts_2012'] + book_set['Checkouts_2013'] + book_set['Checkouts_2014'] + book_set['Checkouts_2015'] + book_set['Checkouts_2016'] + book_set['Checkouts_2017']
book_set['avg checkouts per year'] = book_set['checkouts since pub'] / book_set['years since pub']

In [10]:
book_set.sort_values(by='avg checkouts per year', ascending=False, inplace=True) # sort by avg checkouts per year

In [11]:
book_set = book_set[(book_set['avg checkouts per year'] > 0) & (book_set['avg checkouts per year'] < 100000)] # remove books with no checkouts
book_set = book_set.drop_duplicates(subset=['ISBN']) # remove duplicates
book_set = book_set[book_set['ISBN'].str.len() <= 10] # remove books with invalid ISBNs
subset = book_set[['ISBN', 'Title', 'Author', 'Publisher', 'first_pub', 'Subjects', 'ItemType', 'avg checkouts per year']][:10000]

In [12]:
subset

,ISBN,Title,Author,Publisher,first_pub,Subjects,ItemType,avg checkouts per year
2556759,1594633665,The girl on the train / Paula Hawkins.,"Hawkins, Paula","Riverhead Books, a member of Penguin Group (USA),",2015.0,"Railroad travel Fiction, Witnesses Fiction, Mu...",acbk,1725.000000
731043,0735211205,Into the water / Paula Hawkins.,"Hawkins, Paula","Riverhead Books,",2017.0,"Murder Fiction, Rivers Fiction, Women Crimes a...",acbk,1646.000000
4127854,1476746583,All the light we cannot see : a novel / Anthon...,"Doerr, Anthony, 1973-","Scribner,",2014.0,"Blind Fiction, World War 1939 1945 Youth Franc...",acbk,1634.000000
4167644,0316403431,Today will be different : a novel / Maria Semple.,"Semple, Maria","Little, Brown and Company,",2016.0,"Mothers and sons Fiction, Animators Fiction, F...",acbk,1267.500000
1807585,1524733156,The ministry of utmost happiness / Arundhati Roy.,"Roy, Arundhati","Alfred A. Knopf,",2017.0,"Interpersonal relations Fiction, Self realizat...",pkbknh,1254.000000
...,...,...,...,...,...,...,...,...
21176,1442459905,The tombs of Atuan / Ursula K. Le Guin.,"Le Guin, Ursula K., 1929-","Atheneum Books for Young Readers,",2012.0,"Women priests Juvenile fiction, Robbers and ou...",jcbk,59.666667
2632936,080756320X,Party croc! : a folktale from Zimbabwe / retol...,"MacDonald, Margaret Read, 1940-","Albert Whitman & Company,",2015.0,"Folklore Zimbabwe, Karanga African people Folk...",jcbk,59.666667
1519981,0785161988,"The Avengers. The heroic age / writer, Brian M...","Bendis, Brian Michael","Marvel,",2012.0,Avengers Fictitious characters Comic books str...,acbk,59.666667
547566,0743246985,Wild swans : three daughters of China / Jung C...,"Chang, Jung, 1952-","Touchstone,",2003.0,"Chang Jung 1952, Yang Yu fang 1909 1969, Xia D...",acbk,59.666667


In [13]:
book_set.describe()

,BibNum,Checkouts_2005,Checkouts_2006,Checkouts_2007,Checkouts_2008,Checkouts_2009,Checkouts_2010,Checkouts_2011,Checkouts_2012,Checkouts_2013,Checkouts_2014,Checkouts_2015,Checkouts_2016,Checkouts_2017,first_pub,years since pub,checkouts since pub,avg checkouts per year
count,1.784050e+05,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000,178405.000000
mean,2.436112e+06,2.283540,4.052622,4.947244,6.459819,7.859673,8.513024,9.221692,9.949071,11.702379,11.573594,11.273569,10.971576,8.925675,2005.640576,12.359424,107.733477,15.339093
std,7.683350e+05,12.235697,19.588124,22.266183,28.836582,31.352457,30.973264,30.546673,32.208305,38.056825,34.964541,34.106190,32.691072,28.035548,16.437156,16.437156,249.521621,34.751364
min,1.093000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.000000,1.000000,1.000000,0.001502
25%,2.143037e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2001.000000,5.000000,13.000000,1.238095
50%,2.657285e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2009.000000,9.000000,35.000000,5.100000
75%,2.985855e+06,0.000000,1.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,8.000000,9.000000,9.000000,9.000000,7.000000,2013.000000,17.000000,97.000000,15.000000
max,3.305470e+06,807.000000,1419.000000,1463.000000,1579.000000,1727.000000,2083.000000,1579.000000,1877.000000,3977.000000,2919.000000,2721.000000,2096.000000,1652.000000,2017.000000,1997.000000,8912.000000,1725.000000


In [14]:
subset.describe()

,first_pub,avg checkouts per year
count,10000.00000,10000.000000
mean,2011.59700,117.149482
std,4.31011,90.309404
min,1941.00000,59.666667
25%,2009.00000,71.800000
50%,2012.00000,89.520833
75%,2015.00000,127.000000
max,2017.00000,1725.000000


In [17]:
subset = subset[['ISBN', 'Title', 'Author', 'Publisher', 'first_pub', 'Subjects']]
subset.columns = ['ISBN', 'Title', 'Author', 'Publisher', 'Publication Year', 'Subjects']
subset 


,ISBN,Title,Author,Publisher,Publication Year,Subjects
2556759,1594633665,The girl on the train / Paula Hawkins.,"Hawkins, Paula","Riverhead Books, a member of Penguin Group (USA),",2015.0,"Railroad travel Fiction, Witnesses Fiction, Mu..."
731043,0735211205,Into the water / Paula Hawkins.,"Hawkins, Paula","Riverhead Books,",2017.0,"Murder Fiction, Rivers Fiction, Women Crimes a..."
4127854,1476746583,All the light we cannot see : a novel / Anthon...,"Doerr, Anthony, 1973-","Scribner,",2014.0,"Blind Fiction, World War 1939 1945 Youth Franc..."
4167644,0316403431,Today will be different : a novel / Maria Semple.,"Semple, Maria","Little, Brown and Company,",2016.0,"Mothers and sons Fiction, Animators Fiction, F..."
1807585,1524733156,The ministry of utmost happiness / Arundhati Roy.,"Roy, Arundhati","Alfred A. Knopf,",2017.0,"Interpersonal relations Fiction, Self realizat..."
...,...,...,...,...,...,...
21176,1442459905,The tombs of Atuan / Ursula K. Le Guin.,"Le Guin, Ursula K., 1929-","Atheneum Books for Young Readers,",2012.0,"Women priests Juvenile fiction, Robbers and ou..."
2632936,080756320X,Party croc! : a folktale from Zimbabwe / retol...,"MacDonald, Margaret Read, 1940-","Albert Whitman & Company,",2015.0,"Folklore Zimbabwe, Karanga African people Folk..."
1519981,0785161988,"The Avengers. The heroic age / writer, Brian M...","Bendis, Brian Michael","Marvel,",2012.0,Avengers Fictitious characters Comic books str...
547566,0743246985,Wild swans : three daughters of China / Jung C...,"Chang, Jung, 1952-","Touchstone,",2003.0,"Chang Jung 1952, Yang Yu fang 1909 1969, Xia D..."


In [26]:
subjects = subset.Subjects
subjects

2556759    Railroad travel Fiction, Witnesses Fiction, Mu...
731043     Murder Fiction, Rivers Fiction, Women Crimes a...
4127854    Blind Fiction, World War 1939 1945 Youth Franc...
4167644    Mothers and sons Fiction, Animators Fiction, F...
1807585    Interpersonal relations Fiction, Self realizat...
                                 ...                        
21176      Women priests Juvenile fiction, Robbers and ou...
2632936    Folklore Zimbabwe, Karanga African people Folk...
1519981    Avengers Fictitious characters Comic books str...
547566     Chang Jung 1952, Yang Yu fang 1909 1969, Xia D...
1913379    Montalbano Salvo Fictitious character Fiction,...
Name: Subjects, Length: 10000, dtype: object

In [70]:
from collections import Counter
import re
output = []
count = 0
for subject in subjects:
    if subject == None:
        print('d')
    split = str(subject).split()
    counter = Counter(split)
    most_occuring = counter.most_common(3)
    text = most_occuring[0][0]
    try:
        num = most_occuring[1][1]
        if num > 1:
            text = text + " " + most_occuring[1][0]
    except:
        pass
    try:
        num = most_occuring[2][1]
        if num > 1:
            text = text + " " + most_occuring[2][0]
    except:
        pass
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text).strip()
    output_len = len(output)
    output.append(text)
    if output_len == len(output):
        print('ERROR', subject)
    count += 1
output
count

10000

In [74]:
subset['Subjects'] = output
subset['Publication Year'] = subset['Publication Year'].astype(int)
subset

C:\Users\sethj\AppData\Local\Temp\ipykernel_16500\1278090822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Subjects'] = output
C:\Users\sethj\AppData\Local\Temp\ipykernel_16500\1278090822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Publication Year'] = subset['Publication Year'].astype(int)


,ISBN,Title,Author,Publisher,Publication Year,Subjects
2556759,1594633665,The girl on the train / Paula Hawkins.,"Hawkins, Paula","Riverhead Books, a member of Penguin Group (USA),",2015,Fiction
731043,0735211205,Into the water / Paula Hawkins.,"Hawkins, Paula","Riverhead Books,",2017,Fiction
4127854,1476746583,All the light we cannot see : a novel / Anthon...,"Doerr, Anthony, 1973-","Scribner,",2014,Fiction 1945 France
4167644,0316403431,Today will be different : a novel / Maria Semple.,"Semple, Maria","Little, Brown and Company,",2016,Fiction
1807585,1524733156,The ministry of utmost happiness / Arundhati Roy.,"Roy, Arundhati","Alfred A. Knopf,",2017,Fiction
...,...,...,...,...,...,...
21176,1442459905,The tombs of Atuan / Ursula K. Le Guin.,"Le Guin, Ursula K., 1929-","Atheneum Books for Young Readers,",2012,Juvenile fiction
2632936,080756320X,Party croc! : a folktale from Zimbabwe / retol...,"MacDonald, Margaret Read, 1940-","Albert Whitman & Company,",2015,Folklore African people
1519981,0785161988,"The Avengers. The heroic age / writer, Brian M...","Bendis, Brian Michael","Marvel,",2012,books strips Comic
547566,0743246985,Wild swans : three daughters of China / Jung C...,"Chang, Jung, 1952-","Touchstone,",2003,China Biography


In [78]:
def clean_title(title):
    return title.split(' /')[0]
subset['Title'] = subset['Title'].apply(clean_title)

C:\Users\sethj\AppData\Local\Temp\ipykernel_16500\135987973.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset['Title'] = subset['Title'].apply(clean_title)


In [76]:
subset.to_csv('Exported/SPL_Sel.csv', index=False)

In [77]:
subset

,ISBN,Title,Author,Publisher,Publication Year,Subjects
2556759,1594633665,The girl on the train / Paula Hawkins.,"Hawkins, Paula","Riverhead Books, a member of Penguin Group (USA),",2015,Fiction
731043,0735211205,Into the water / Paula Hawkins.,"Hawkins, Paula","Riverhead Books,",2017,Fiction
4127854,1476746583,All the light we cannot see : a novel / Anthon...,"Doerr, Anthony, 1973-","Scribner,",2014,Fiction 1945 France
4167644,0316403431,Today will be different : a novel / Maria Semple.,"Semple, Maria","Little, Brown and Company,",2016,Fiction
1807585,1524733156,The ministry of utmost happiness / Arundhati Roy.,"Roy, Arundhati","Alfred A. Knopf,",2017,Fiction
...,...,...,...,...,...,...
21176,1442459905,The tombs of Atuan / Ursula K. Le Guin.,"Le Guin, Ursula K., 1929-","Atheneum Books for Young Readers,",2012,Juvenile fiction
2632936,080756320X,Party croc! : a folktale from Zimbabwe / retol...,"MacDonald, Margaret Read, 1940-","Albert Whitman & Company,",2015,Folklore African people
1519981,0785161988,"The Avengers. The heroic age / writer, Brian M...","Bendis, Brian Michael","Marvel,",2012,books strips Comic
547566,0743246985,Wild swans : three daughters of China / Jung C...,"Chang, Jung, 1952-","Touchstone,",2003,China Biography
